## Построение упрощенного дерева

Проблема построения филогенетического дерева для большой выборки стоит в вычислительных мощностях и в не удобстве анализирования. Возможным решением взять 10% от всех последовательностей, имеющие максимальный e-value

In [6]:
import pandas as pd

In [187]:
data = pd.read_csv('With_Cys_for tree/protisits.csv')

In [188]:
data.drop('Unnamed: 0', axis = 1, inplace = True)
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan,kingdom
0,A0A0L0FZB2,24,120,15,121,PF13426.7,LOV,Domain,7,103,104,53.7,2.000000e-14,1,CL0183,Eukaryota Ichthyosporea
1,A0A126X2D8,360,402,357,404,PF07716.15,bZIP_2,Family,6,48,54,28.2,1.400000e-06,1,CL0018,Eukaryota Stramenopiles
2,A0A126X2D8,521,611,513,616,PF13426.7,LOV,Domain,7,97,104,54.1,1.500000e-14,1,CL0183,Eukaryota Stramenopiles
3,A0A126WYE7,241,327,232,336,PF13426.7,PAS_9,Domain,8,94,104,24.7,2.100000e-05,1,CL0183,Eukaryota Stramenopiles
4,A0A126WYE7,472,595,472,596,PF00615.19,RGS,Domain,1,117,118,25.7,1.100000e-05,1,CL0272,Eukaryota Stramenopiles


In [193]:
d = data[(data['hmm name'] == 'LOV')].sort_values('E-value')

Для архей отсев прошел по границе e-value < 1e-23

Для бактерий e-value < 1e-22

Для грибов e-value < 1e-17

Для растений e-value < 1e-20

Для протист e-value < 1e-17

In [196]:
d = d[d['E-value'] < 1e-17]

In [197]:
names = d['seq id']

In [198]:
data = data.loc[data['seq id'].isin(list(names))]

In [199]:
tfile = open('With_Cys_for tree/protisits.csv', 'w')
tfile.write(data.to_csv())
tfile.close()

In [195]:
d.shape

(21, 16)

Собираем файл фаста для выравнивания, не забываем, что необходим так же формат PHYLIP, поэтому имена должный быть не больше 10 знаков, составим общую csv таблицу


In [3]:
import glob, os
os.chdir("/home/dmelikhova/projects/190115_cysless_lov/With_Cys_for_tree/")
files = glob.glob("*.csv")

In [26]:
files.pop(0)
files.pop(0)
files.pop(2)
files.pop(4)

'domains_LOV_1_new.csv'

In [4]:
files = ['domains_LOV_1_cl.csv', 'domains_LOV_2_cl.csv', 'domains_LOV_3_cl.csv', 'domains_LOV_4_cl.csv']

In [7]:
data = pd.DataFrame(columns=['seq id', 'alignment start', 'alignment end', 'envelope start', 'envelope end',
                            'hmm acc', 'hmm name', 'type', 'hmm start', 'hmm end', 'hmm length', 'bit score',
                            'E-value', 'significance', 'clan', 'kingdom'])


In [8]:
for file in files:
    d = pd.read_csv('/home/dmelikhova/projects/190115_cysless_lov/' + file)
    data = pd.concat([data, d])

In [9]:
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan,kingdom
0,A0A0N0BRB8,18,105,14,116,PF00072.24,Response_reg,Domain,14,102,112,45.5,7.100000e-12,1,CL0304,Archaea Euryarchaeota
1,A0A0N0BRB8,152,256,150,256,PF13426.7,LOV,Domain,3,104,104,73.3,1.500000e-20,1,CL0183,Archaea Euryarchaeota
2,A0A0N0BRB8,304,435,299,436,PF13185.6,GAF_2,Domain,6,137,138,36.8,4.300000e-09,1,CL0161,Archaea Euryarchaeota
3,A0A0N0BRB8,447,600,447,600,PF15915.5,BAT,Domain,1,156,156,119.6,1.100000e-34,1,No_clan,Archaea Euryarchaeota
4,A0A0N0BRB8,606,658,606,658,PF04967.12,HTH_10,Domain,1,53,53,71.5,3.600000e-20,1,CL0123,Archaea Euryarchaeota


In [37]:
tfile = open('/home/dmelikhova/projects/190115_cysless_lov/With_Cys_for_tree/data_for_tree.csv', 'w')
tfile.write(data.to_csv())
tfile.close()

In [10]:
data = data.reset_index(drop=True)

In [1]:
with open('/home/dmelikhova/projects/190115_cysless_lov/final_proteins_cl.fasta','r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line.lstrip('>')
        else:
            fasta.setdefault(name, []).append(line)

In [11]:
data_LOV = data[data['hmm name'] == 'LOV'].reset_index(drop = True)
data_TLOV = data[data['hmm name'] == 'TLOV'].reset_index(drop = True)
data_LOV_ = data[data['hmm name'] == '_LOV'].reset_index(drop = True)

In [12]:
new_fasta = {}
for k,v in fasta.items():
    if k in list(data_LOV['seq id']):
        for n in range(data_LOV.loc[data_LOV['seq id'] == k, 'envelope start'].shape[0]): 
            i, j = (int(data_LOV.loc[data_LOV['seq id'] == k, 'envelope start'].reset_index(drop = True)[n]), 
                    int(data_LOV.loc[data_LOV['seq id'] == k, 'envelope end'].reset_index(drop = True)[n]))
            LOV = ''.join(v)[i-1: j]
            new_fasta['>' + k + str(n)] = LOV
for k,v in fasta.items():
    if k in list(data_TLOV['seq id']):
        i, j = (int(data_TLOV.loc[data_TLOV['seq id'] == k, 'envelope start'].reset_index(drop = True)[n]), 
                 int(data_TLOV.loc[data_TLOV['seq id'] == k, 'envelope end'].reset_index(drop = True)[n]))
        LOV = ''.join(v)[i-1: j]
        new_fasta['>' + k + 'T1'] = LOV
for k,v in fasta.items():
    if k in list(data_LOV_['seq id']):
        i, j = (int(data_LOV_.loc[data_LOV_['seq id'] == k, 'envelope start'].reset_index(drop = True)[n]), 
                int(data_LOV_.loc[data_LOV_['seq id'] == k, 'envelope end'].reset_index(drop = True)[n]))
        LOV = ''.join(v)[i-1: j]
        new_fasta['>' + k + 'T2'] = LOV

In [159]:
for k in new_fasta.keys():
    if len(k) > 11:
        new_fasta['>'+ k[3:]] = new_fasta.pop(k)

In [160]:
for k in new_fasta.keys():
    if len(k) > 11:
        print(k)

In [158]:
new_fasta['>A1Z4HNP10'] = new_fasta.pop('>A0A1Z4HNP10')

In [67]:
'>A0A1M5G3X30'[3:]

'A1M5G3X30'

In [13]:
len(new_fasta)

220

In [14]:
new_file = []
for k, v in new_fasta.items():
    new_file.append(k + '\n')
    new_file.append(v + '\n')

In [15]:
with open('/home/dmelikhova/projects/190115_cysless_lov/tree_cl_not_phylip.fasta', 'w') as f:
    f.write(''.join(new_file))

In [125]:
len(new_fasta)

0

Раскрасим в полученнном дереве цистеиновые остатки

In [164]:
#tree_cl содержит последовательности с измененными именами для безцистеиновых ловов
with open('/home/dmelikhova/projects/190115_cysless_lov/tree_cl.fasta','r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line.lstrip('>')
        else:
            fasta.setdefault(name, []).append(line)

In [167]:
for k in fasta.keys():
    print(k + '\t' + 'red\ttoroot')

A285MUA00	red	toroot
A1X4GBI90	red	toroot
A2D8CZZ10	red	toroot
M0D0B10	red	toroot
A252DKD40	red	toroot
A1G7KVI50	red	toroot
M0EVT30	red	toroot
A101D4P00	red	toroot
A126WX770	red	toroot
A4TY700	red	toroot
M1XS320	red	toroot
A2P7R4D50	red	toroot
A2R6F6430	red	toroot
A2W4P6990	red	toroot
A0C2YV270	red	toroot
C6HXY90	red	toroot
B0R5N60	red	toroot
A2T1KTE50	red	toroot
A1Z4UFN50	red	toroot
A1Z4KTG30	red	toroot
M0DQM90	red	toroot
A126WVX20	red	toroot
A250KTS60	red	toroot
A256HJZ80	red	toroot
A086WFV90	red	toroot
M0PQ370	red	toroot
A256I8X30	red	toroot
A0U3IE240	red	toroot
M0NUF30	red	toroot
A1F4A9A40	red	toroot
U7G3B40	red	toroot
A256H8K60	red	toroot
A1I7LB450	red	toroot
A1Z4GNL10	red	toroot
A238W3F50	red	toroot
A256IHV40	red	toroot
A0D5N8D70	red	toroot
A1H3HFD70	red	toroot
A2T2V0L60	red	toroot
A2S8ESS10	red	toroot
A0U5MKD50	red	toroot
A068YUC80	red	toroot
A256IB300	red	toroot
A2T1DAT10	red	toroot
A256II670	red	toroot
A1H1QY270	red	toroot
A1I3IR900	red	toroot
A256HIR10	red	toroot
A0N9MTI70	re

In [168]:
from ete3 import PhyloTree, TreeStyle

ImportError: cannot import name 'TreeStyle'